In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv').values

# Data Preprocessing
X = data[:, 1:] / 255.0
Y = data[:, 0]

# Split the data into train and dev sets
X_train, X_dev, Y_train, Y_dev = train_test_split(X, Y, test_size=1000, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_dev = torch.tensor(X_dev, dtype=torch.float32)
Y_dev = torch.tensor(Y_dev, dtype=torch.long)

# Data Augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

class MNISTDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx].reshape(28, 28)
        label = self.Y[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = MNISTDataset(X_train, Y_train, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(MNISTDataset(X_dev, Y_dev), batch_size=batch_size)

# Build the 3-layered classifier using PyTorch
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Initialize the model and move it to the GPU if available
model = Classifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
def train(model, train_loader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Training the model
train(model, train_loader, criterion, optimizer, device)

# Evaluate the model on the dev set
def evaluate(model, data_loader, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
    accuracy = correct_predictions / total_predictions
    return accuracy

accuracy_dev = evaluate(model, dev_loader, device)
print(f"Dev Accuracy: {accuracy_dev:.4f}")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Epoch 1/20, Loss: 1.275467904328184
Epoch 2/20, Loss: 0.750639018793002
Epoch 3/20, Loss: 0.49308091953001604
Epoch 4/20, Loss: 0.4028512376147016
Epoch 5/20, Loss: 0.3546738626316259
Epoch 6/20, Loss: 0.32251493711832346
Epoch 7/20, Loss: 0.29575671880925875
Epoch 8/20, Loss: 0.2784094947583497
Epoch 9/20, Loss: 0.2616001234365142
Epoch 10/20, Loss: 0.24877936087909058
Epoch 11/20, Loss: 0.24044202601714365
Epoch 12/20, Loss: 0.22795777642364248
Epoch 13/20, Loss: 0.22043852673119949
Epoch 14/20, Loss: 0.21194397740054427
Epoch 15/20, Loss: 0.2061938919743026
Epoch 16/20, Loss: 0.19999620131276513
Epoch 17/20, Loss: 0.19480856146239175
Epoch 18/20, Loss: 0.19221777146952387
Epoch 19/20, Loss: 0.18669812677954084
Epoch 20/20, Loss: 0.18316924979680693
Dev Accuracy: 0.9460


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Read the data
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv').values

# Data Preprocessing
X = data[:, 1:] / 255.0
Y = data[:, 0]

# Split the data into train, dev, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_dev = torch.tensor(X_dev, dtype=torch.float32)
Y_dev = torch.tensor(Y_dev, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)

# Data Augmentation
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

transform_dev = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

class MNISTDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx].reshape(28, 28)
        label = self.Y[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

train_dataset = MNISTDataset(X_train, Y_train, transform=transform_train)
dev_dataset = MNISTDataset(X_dev, Y_dev, transform=transform_dev)
test_dataset = MNISTDataset(X_test, Y_test, transform=transform_dev)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Build the 3-layered classifier using PyTorch
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Initialize the model and move it to the GPU if available
model = Classifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Training loop
def train(model, train_loader, criterion, optimizer, device, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")

# Training the model
train(model, train_loader, criterion, optimizer, device)

# Evaluate the model on the dev and test sets
def evaluate(model, data_loader, device):
    model.eval()
    all_labels = []
    all_preds = []
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return running_loss / len(data_loader), accuracy, precision, recall, f1

# Evaluation on dev set
dev_loss, dev_accuracy, dev_precision, dev_recall, dev_f1 = evaluate(model, dev_loader, device)
print(f"Dev Loss: {dev_loss:.4f}, Dev Accuracy: {dev_accuracy:.4f}, Dev Precision: {dev_precision:.4f}, Dev Recall: {dev_recall:.4f}, Dev F1: {dev_f1:.4f}")

# Evaluation on test set
test_loss, test_accuracy, test_precision, test_recall, test_f1 = evaluate(model, test_loader, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}, Test F1: {test_f1:.4f}")


Epoch 1/20, Loss: 1.350054339681353
Epoch 2/20, Loss: 0.8604557983648209
Epoch 3/20, Loss: 0.5985136251222519
Epoch 4/20, Loss: 0.4606383996066593
Epoch 5/20, Loss: 0.39678626699107034
Epoch 6/20, Loss: 0.3574375542998314
Epoch 7/20, Loss: 0.3319618473166511
Epoch 8/20, Loss: 0.30325466590268274
Epoch 9/20, Loss: 0.28980978308689026
Epoch 10/20, Loss: 0.2783087069434779
Epoch 11/20, Loss: 0.2685840334494909
Epoch 12/20, Loss: 0.25681485570612406
Epoch 13/20, Loss: 0.2442877465912274
Epoch 14/20, Loss: 0.2374122943267936
Epoch 15/20, Loss: 0.22928646819222542
Epoch 16/20, Loss: 0.2207986860970656
Epoch 17/20, Loss: 0.22124784427029745
Epoch 18/20, Loss: 0.21448723951265924
Epoch 19/20, Loss: 0.20449892723134586
Epoch 20/20, Loss: 0.20467381651912417
Dev Loss: 0.1682, Dev Accuracy: 0.9467, Dev Precision: 0.9461, Dev Recall: 0.9456, Dev F1: 0.9455
Test Loss: 0.1759, Test Accuracy: 0.9436, Test Precision: 0.9440, Test Recall: 0.9418, Test F1: 0.9423
